In [2]:
from queue import PriorityQueue

In [30]:
class Library:
    def __init__(self, id, n_books, signup_time, books_per_day, books):
        self.id = id
        self.n_books = n_books                  # number of books
        self.signup_time = signup_time          # signup time
        self.books_per_day = books_per_day      # books per day
        self.books = books
    
    def __str__(self):
        return f"{self.id} {self.n_books} {self.signup_time} {self.books_per_day} {self.books}"
    
    def calculate_score_per_day(self):
        score = 0
        green_books = 0
        pq = PriorityQueue()
        while (not self.books.empty()):
            book = self.books.get()
            if (book.status == 0):
                score += book.score
                green_books += 1
                pq.put(book)
            else:
                break
        
        while (not pq.empty()):
            self.books.put(pq.get())
        
        if (green_books == 0):
            return (0, 0)
        
        time_taken = self.signup_time + green_books / self.books_per_day

        return (score * self.books_per_day / (green_books * time_taken), green_books)
        

    def __lt__(self, other):
        spd1 = self.calculate_score_per_day()
        spd2 = other.calculate_score_per_day()
        if (spd1[0] == spd2[0]):
            return spd1[1] > spd2[1]
        return spd1[0] > spd2[0]

class Book:
    def __init__(self, id, score):
        self.id = id
        self.score = score
        self.status = 0
    
    def __str__(self):
        return f"{self.id} {self.score}" 
    
    def __lt__(self, other):
        if (self.status == other.status):
            return self.score > other.score
        return self.status < other.status
    
    def turn_yellow(self):
        self.status = 1

    def turn_red(self):
        self.status = 2
    
class Singleton:
    def __init__(self, total_books, total_libs, total_days, scores, libs):
        self.total_books = total_books  # total number of books
        self.total_libs = total_libs  # total number of libraries
        self.total_days = total_days  # total number of days
        self.scores = scores
        self.libs = libs

    def __str__(self):
        return f"{self.total_books} {self.total_libs} {self.total_days} {self.libs}"


In [36]:
libs_dict = {}
books_dict = {}

def read_file(filename):

    f = open(filename, "r")

    line = f.readline()

    values = [int(x) for x in line.split()]

    total_books = values[0]
    total_libs = values[1]
    total_days = values[2]

    line = f.readline()
    scores = [int(x) for x in line.split()]

    for i in range(len(scores)):
        books_dict[i] = Book(i, scores[i])

    libs = PriorityQueue()
    i = 0
    while True:
        line = f.readline()    
        values = [int(x) for x in line.split()]

        if len(values) == 0:
            break
        
        line = f.readline()
        books = [int(x) for x in line.split()]

        q = PriorityQueue()
        for b in books:
            q.put(books_dict[b])

        lib = Library(i, values[0], values[1], values[2], q)
        
        libs.put(lib)
        libs_dict[i] = lib
        i += 1

    f.close()

    return total_books, total_libs, total_days, scores, libs

In [55]:
def calculate_score(solution, total_days):
    score = 0
    days = 0
    scanned_books = set()
    libs_signed_up = set()
    
    for tup in solution:
        lib = libs_dict[tup[0]]

        if (lib.id in libs_signed_up):
            return -1

        days += lib.signup_time

        if (days >= total_days):
            break

        days_left = total_days - days
        books_left = days_left * lib.books_per_day
        books = tup[1][:books_left]
        
        for b in books:
            if (b not in scanned_books):
                score += books_dict[b].score
                scanned_books.add(b)
        
        libs_signed_up.add(lib.id)
    
    return score


In [44]:
total_books, total_libs, total_days, scores, libs = read_file("data/a_example.txt")
total_books, total_libs, total_days, scores, libs= read_file("data/b_read_on.txt")
#total_books, total_libs, total_days, scores, libs = read_file("data/c_incunabula.txt")
#total_books, total_libs, total_days, scores, libs = read_file("data/d_tough_choices.txt")
#total_books, total_libs, total_days, scores, libs = read_file("data/e_so_many_books.txt")
#total_books, total_libs, total_days, scores, libs = read_file("data/f_libraries_of_the_world.txt")

att = Singleton(total_books, total_libs, total_days, scores, libs)

"""
while (not att.libs.empty()):
    lib = att.libs.get()
    print(lib.calculate_score_per_day())
    while not lib.books.empty():
        print(lib.books.get())
"""



0


In [48]:
def solve(att):
    solution = []
    days = 0
    scanned_books = set()
    libs_signed_up = set()
    while (not att.libs.empty()):
        lib = att.libs.get()
        days += lib.signup_time
        if (days >= att.total_days):
            break
        days_left = att.total_days - days
        books_left = days_left * lib.books_per_day
        books = []
        while (not lib.books.empty()):
            book = lib.books.get()
            if (book.status == 0):
                books.append(book.id)
                book.turn_red()
                scanned_books.add(book.id)
            
            if (len(books) == books_left):
                break
        
        solution.append((lib.id, books))
        libs_signed_up.add(lib.id)

    return solution
            


In [49]:
solution = solve(att)
print(calculate_score(solution, att.total_days))

KeyboardInterrupt: 

In [51]:
data_files = [
    "data/a_example.txt",
    "data/b_read_on.txt",
    "data/c_incunabula.txt",
    "data/d_tough_choices.txt",
    "data/e_so_many_books.txt",
    "data/f_libraries_of_the_world.txt"
]

atts = []



File data/a_example.txt ingested
File data/b_read_on.txt ingested
File data/c_incunabula.txt ingested
File data/d_tough_choices.txt ingested
File data/e_so_many_books.txt ingested
File data/f_libraries_of_the_world.txt ingested


In [56]:
final_score = 0

for file in data_files:
    libs_dict = {}
    books_dict = {}
    total_books, total_libs, total_days, scores, libs = read_file(file)
    att = Singleton(total_books, total_libs, total_days, scores, libs)
    print(f"File {file} ingested")

    total_days = att.total_days
    solution = solve(att)
    solution_score = calculate_score(solution, total_days)
    final_score += solution_score
    
    print(f"Solution for file {file}: {solution}")
    print(f"Score: {solution_score}")

print(f"Final score: {final_score}")

File data/a_example.txt ingested
Solution for file data/a_example.txt: [(0, [3, 4, 2, 1, 0]), (1, [5])]
Score: 21
File data/b_read_on.txt ingested
Solution for file data/b_read_on.txt: [(38, [38000, 38111, 38897, 38494, 38950, 38038, 38108, 38242, 38515, 38495, 38114, 38946, 38244, 38042, 38514, 38496, 38117, 38005, 38246, 38498, 38046, 38120, 38013, 38248, 38497, 38050, 38123, 38020, 38250, 38008, 38054, 38126, 38025, 38252, 38002, 38014, 38030, 38254, 38998, 38999, 38997, 38996, 38992, 38993, 38991, 38990, 38987, 38989, 38986, 38985, 38982, 38984, 38981, 38980, 38977, 38979, 38976, 38975, 38974, 38973, 38972, 38971, 38970, 38969, 38968, 38965, 38967, 38964, 38961, 38963, 38960, 38957, 38959, 38956, 38953, 38955, 38952, 38949, 38951, 38948, 38945, 38947, 38944, 38943, 38942, 38941, 38940, 38939, 38938, 38937, 38936, 38935, 38934, 38933, 38932, 38931, 38930, 38929, 38928, 38927, 38926, 38925, 38924, 38923, 38922, 38921, 38920, 38919, 38918, 38917, 38916, 38915, 38914, 38913, 38912, 389

In [34]:
print(calculate_score([(0, [3, 4, 2, 1, 0]), (1, [5])], 7))

0
